In [ ]:
import os
import dateutil.parser

import requests

from bs4 import BeautifulSoup

import pandas as pd


In [ ]:
def monetary_to_int(monetary):
    monetary = monetary.replace('$', '').replace(',', '')
    return int(monetary)

def runtime_to_minutes(runtime):
    runtime = runtime.split()
    try:
        minutes = int(runtime[0])*60 + int(runtime[2])
        return minutes
    except:
        return None

def to_date(date):
    date = dateutil.parser.parse(date)
    return date


In [ ]:
def get_top_grossers(offset):
    
    url = f'https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?offset={offset}'

    resp = requests.get(url)
    
    assert resp.status_code == 200, f'No response from url: {url}.'
    
    soup = BeautifulSoup(resp.text, 'html.parser')
    
    table = soup.find('table')
    
    rows = [row for row in table.find_all('tr')]
    
    movies = {}

    for row in rows[1:]:
        items = row.find_all('td')
        rank = items[0].text
        link = items[1].find('a')
        title, url = link.text, link['href']
        worldwide_gross = monetary_to_int(items[2].text)
        domestic_gross = monetary_to_int(items[3].text)
        foreign_gross = monetary_to_int(items[5].text)
        year = to_date(items[7].text)
        print(year, type(year))

        '''
                               
        movies[rank] = [i.text for i in items[1:]] + [url]

    headers = ['title', 'worldwide_gross', 'domestic_gross', 'domestic_pct', 'international_gross', 'international_pct', 'year', 'link_imdb']
    
    movies_df = pd.DataFrame(movies).T
    movies_df.columns = headers
    
    return movies_df
    '''
    

In [ ]:
df = get_top_grossers(0)

In [ ]:
df.head()

In [ ]:
filename = 'top_gross_films.csv'
offsets = [0,200,400]
for num in offsets:
    df = get_top_grossers(num)
    
    if not os.path.exists(filename):
        df.to_csv(filename)
    
    elif os.path.exists(filename):
        df.to_csv(filename, mode='a', header=False)